In [1]:
#error function: calculuate cross entropy
#mean squared error is used for linear regression

#weights: relative importances of features
#bais: "shift"/location of the activation function
#activiation function is sigmoid

import numpy as np

In [ ]:
#learning rate for gradient descent
LR = 0.0015

#number of random starting points to take
n_iters = 100

#number of steps to take in GD
n_steps = 1000

#feature inputs -> dotted weights, + bias -> y = sigmoid(x) -> find_loss(y)

#we can write in terms of wegiths and biases (substitute into the find loss function)
#feature inputs -> dotted weights, + bias -> y = sigmoid(x) -> find_loss(y) -> 

weights, biases = 0, 0

In [ ]:
#Tanh, ReLU (less computation time)
#Why choose sigmoid?

#Activation function: 
def sigmoid(x):
    """
    Computes sigmoid function to calculate probability based on single-value output of logistic regression
    and predictions. 
    Input: Computed prediction value
    Returns: scalar probability between (0, 1)
    """
    return 1.0/(1.0 + np.exp(-x))

def cross_entropy(predictions, targets):
    """
    Computes cross entropy between predicted probabilities and actual 0/1 classifications. 
    Input: predictions vector of predicted classes' probabilities
           targets vector of actual 0/1 classes
    Returns: scalar representing loss
    """
    n = predictions.shape[0]
    return -(1/n) * np.sum(targets*np.log(predictions) + (1-targets)*np.log(1-predictions))

#E'(weights, biases)
#Other activation function and their derivatives given here: https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6
def calc_dir(weights, bias, predicted, actual):
    """
    Computes cross entropy between targets (encoded as one-hot vectors)
    and predictions. 
    Input: weights = coefficients for features
           bias = single-value added to prediction output     
           predicted = predicted 0/1 classes for classification output
           actual = actual 0/1 classes for classification output
    Returns: tuple of 2 elements
             gradient descent vector <dw, db> = vector of greatest descent for weights and biases, respectively
    """
    N = len(weights)
    dW = (1/N) * 2 * np.dot(weights.T, (predicted - actual))
    dB = (1/N) * 2 * np.sum(predicted - actual)
    
    return [dW, dB]

In [ ]:
def gradient_descent(weights, bias, x, y, LR):
    """
    Performs gradient descent to optimize initial starting weight and biases
    
    Input: x = training set containing input features
           y = training set containing expected classification
           weights = n-d vector/numpy_array
           bias = scalar
           LR = scalar, learning rate
    Returns: tuple of 2 elements, (weights, bias)
    """
    
    for i in n_steps:
        #for each row of features, we generate a corresponding number by multiplying each weight by feature value
        predicted_vals = np.dot(weights.T, x) + bias
        print(predicted_vals.shape)
        
        #for each feature value, we predict its class probability utilizing the sigmoid function
        predicted_probs = sigmoid(predicted_vals)
        print(predicted_probs.shape)
        
        print("On step " + str(i) + "th step, loss equals " + cross_entropy(predicted_probs, y))
        
        #calculated derivatives
        derivs = calc_dir(weights, bias, predicted_probs)
        dW = derivs[0], dB = derivs[1]
        
        #"descent" down
        weights -= LR * dW
        bias -= LR * dB
        
    return [weights, bias]

In [ ]:
def train(x, y):
    """
    Optimizes weights and biases given initial training set. Utilizes entire training set (must train-test-split beforehand)
    
    Input: x = model input of training dataset, 2darray containing m samples (entries) and n features
           y = expected model output of training dataset, 1darray containing n classification results (0/1)

    """
    global weights, bias
    m, n = x.shape
    
    #random starting points: vector/np_array of size n
    weights = np.random.rand(n)
    #random starting points: random bias
    bias = np.random.rand(1)[0]
    
    res = gradient_descent(weights, bias, x, y, LR)
    weights = res[0], bias = res[1]
    
    #Start at a random location
#     for _ in n_iters:
        
        

In [ ]:
def predict_single_prob(features);
    """
    Utilizes pre-optimized weights and biases to outputs class probability given nd-vector of features
    
    Input: nd-vector of feature
    Output: scalar, class probability of features
    """

    predicted_val = np.dot(weights.T, x) + bias
    predicted_prob = sigmoid(predicted_vals)
    
    return predicted_prob

def predict_probs(x):
    """
    Utilizes pre-optimized weights and biases to outputs class probability given m entries of nd-vector of features
    
    Input: m by n matrix of feature
    Output: m-d vector of class probabilities for each entry
    """
    return predict_single_prob(x)

def predict(x):
    """
    Utilizes pre-optimized weights and biases to outputs class given m entries of nd-vector of features
    
    Input: m by n matrix of feature
    Output: m-d vector of 0/1 class for each entry
    """
    return round(predict_probs(x))